<a href="https://colab.research.google.com/github/cmyHuawei/NeMoAudioCodec/blob/main/Copy_of_Audio_Codec_Training_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# License

> Copyright (c) 2024, NVIDIA CORPORATION & AFFILIATES. All rights reserved.
>
> Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
>
> http://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Introduction

In this tutorial we show how to use NeMo to train and fine-tune **neural audio codecs**.

Neural audio codecs are deep learning models that compress audio into a low bitrate representation. The compact embedding space created by these models can be useful for various speech tasks, such as TTS and ASR.

<div>
<img src="https://github.com/NVIDIA/NeMo/releases/download/v1.22.0/nemo_audio_codec.png" width="800", height="400"/>
</div>

Audio codec models typically have an *encoder-quantizer-decoder* structure. The **encoder** takes an input audio signal and encodes it into a sequence of embeddings. The **quantizer** discretizes the embeddings to create a lookup table known as a **codebook**. The embeddings saved in the codebook are referred to as **audio codes**. The **decoder** takes the audio codes as input and attempts to reconstruct the original audio signal.

To store compressed audio we only need to save the codebook index for each embedding in an audio sequence. This is how audio codec models achieve low bitrates. The codebook indices for an audio are referred to **audio tokens**. It is becoming common for speech generation models to synthesize speech by predicting audio tokens.

In NeMo we have implementations of the [SEANet encoder and decoder](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/tts/modules/encodec_modules.py#L146)  used by [EnCodec](https://github.com/facebookresearch/encodec). As well as a [ResNet encoder](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/tts/modules/audio_codec_modules.py#L1035) and [HiFi-GAN decoder](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/tts/modules/audio_codec_modules.py#L875). For quantizers we support [Residual Vector Quantizer](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/tts/modules/encodec_modules.py#L694) (**RVQ**) and [Finite Scalar Quantizer](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/tts/modules/audio_codec_modules.py#L409) (**FSQ**).


# Install

In [ ]:
BRANCH = 'main'
# Install NeMo library. If you are running locally (rather than on Google Colab), comment out the below line
# and instead follow the instructions at https://github.com/NVIDIA/NeMo#Installation
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[tts]

DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[tts] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-gy46w1wj/nemo-toolkit_ebe757311d8a4055ad7e34387692e9f9
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-gy46w1wj/nemo-toolkit_ebe757311d8a4055ad7e34387692e9f9
  Resolved https://github.com/NVIDIA/NeMo.git to commit fe877b68e473adafb99a276abd21b0ebf63bdf54
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4

In [ ]:
from pathlib import Path

In [ ]:
# !pip install ipdb # C - A

In [ ]:
# import ipdb # C - A

In [ ]:
# Directory where tutorialscripts will run and outputs will be saved.
ROOT_DIR = Path().absolute() / "codec_tutorial"

# Nemo code paths
NEMO_DIR = ROOT_DIR / "nemo"
NEMO_SCRIPT_DIR = NEMO_DIR / "scripts" / "dataset_processing" / "tts"
NEMO_EXAMPLES_DIR = NEMO_DIR / "examples" / "tts"
NEMO_CONFIG_DIR = NEMO_EXAMPLES_DIR / "conf"

nemo_download_dir = str(NEMO_DIR)
# Download local version of NeMo scripts. If you are running locally and want to use your own local NeMo code,
# comment out the below line and set NEMO_ROOT_DIR to your local path.
!git clone -b $BRANCH https://github.com/NVIDIA/NeMo.git $nemo_download_dir

Cloning into '/content/codec_tutorial/nemo'...
remote: Enumerating objects: 202716, done.
remote: Counting objects: 100% (1775/1775), done.
remote: Compressing objects: 100% (769/769), done.
remote: Total 202716 (delta 1458), reused 1059 (delta 1006), pack-reused 200941 (from 3)
Receiving objects: 100% (202716/202716), 273.43 MiB | 23.93 MiB/s, done.
Resolving deltas: 100% (152397/152397), done.


# Configuration

Predefined model configurations are available in https://github.com/NVIDIA/NeMo/tree/main/examples/tts/conf/audio_codec.

Configurations available include:

*   **audio_codec_*.yaml**: Audio codec configurations optimized for various sampling rates.
*   **mel_codec_*.yaml**: A mel-spectrogram based codec designed to maximize the performance of speech synthesis models.
*   **encodec_*.yaml**: A reproduction of the original [EnCodec](https://arxiv.org/abs/2210.13438) model setup.
*   **audio_codec_low_frame_rate_22050.yaml**: The [Low Frame-rate Speech Codec](https://arxiv.org/abs/2409.12117): an audio codec that achieves high-quality audio compression with a 1.89 kbps bitrate and 21.5 frames per second.

This tutorial can be run with any of our predefined configs. As a default we have selected `audio_codec_16000.yaml`, which works for 16kHz audio.

In [ ]:
from omegaconf import OmegaConf

In [ ]:
# CONFIG_FILENAME = "audio_codec_22050.yaml" # Original
CONFIG_FILENAME = "audio_codec_low_frame_rate_22050.yaml" # C - M
CONFIG_DIR = NEMO_CONFIG_DIR / "audio_codec"

config_filepath = CONFIG_DIR / CONFIG_FILENAME

if not config_filepath.exists():
  raise ValueError(f"Config file does not exist {config_filepath}")

In [ ]:
# Read model name and sample rate from model configuration
omega_conf = OmegaConf.load(config_filepath)
MODEL_NAME = omega_conf.name
SAMPLE_RATE = omega_conf.sample_rate
print(f"Training {MODEL_NAME} with sample rate {SAMPLE_RATE}")

Training AudioCodec with sample rate 22050


We provide pretrained model checkpoints for fine-tuning.

A list of models available on NGC can be found [here](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/tts/models/audio_codec.py#L645).

A list of models available on Hugging Face can be found [here](https://huggingface.co/collections/nvidia/nemo-audio-codecs-674f57ab6cb1324f997b5d5b). To use a checkpoint from hugging face, add "nvidia/" before the model name.

In [ ]:
# cls.list_available_models() # C - A. Wrong

NameError: name 'cls' is not defined

In [ ]:
from nemo.collections.tts.models.audio_codec import AudioCodecModel

# pretrained_model_name = "nvidia/audio-codec-22khz" # Original
# pretrained_model_name = "audio_codec_16khz_small" # C - M: For WavLM (low frame-rate paper). Wrong. The model names are available in NeMo_Nvidia\docs\source\tts\data\models_codec.csv
# pretrained_model_name = "audio_codec_low_frame_rate_22khz" # C - M: For WavLM (low frame-rate paper). Typo in the name. The model name is obtained from https://huggingface.co/nvidia/low-frame-rate-speech-codec-22khz/blob/1d7c8f669f231249b7183c910726102d1c94e650/README.md#training
pretrained_model_name = "nvidia/low-frame-rate-speech-codec-22khz" # C - M: For WavLM (low frame-rate paper). The model names are available in NeMo_Nvidia\docs\source\tts\data\models_codec.csv

if pretrained_model_name is None:
  MODEL_CHECKPOINT_PATH = None
else:
  MODEL_CHECKPOINT_PATH = AudioCodecModel.from_pretrained(model_name=pretrained_model_name, return_model_file=True)

low-frame-rate-speech-codec-22khz.nemo:   0%|          | 0.00/619M [00:00<?, ?B/s]

# Dataset Preparation

For our tutorial, we use a subset of [VCTK](https://datashare.ed.ac.uk/handle/10283/2950) dataset with 5 speakers (p225-p229).

In [ ]:
import tarfile
import wget

from nemo.collections.asr.parts.utils.manifest_utils import read_manifest, write_manifest

In [ ]:
# Create dataset directory
DATA_DIR = ROOT_DIR / "data"

DATA_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# Download the dataset
dataset_url = "https://vctk-subset.s3.amazonaws.com/vctk_subset_multispeaker.tar.gz"
dataset_tar_filepath = DATA_DIR / "vctk.tar.gz"

if not dataset_tar_filepath.exists():
    wget.download(dataset_url, out=str(dataset_tar_filepath))

In [ ]:
# Extract the dataset
with tarfile.open(dataset_tar_filepath) as tar_f:
    tar_f.extractall(DATA_DIR)

In [ ]:
DATASET_DIR = DATA_DIR / "vctk_subset_multispeaker"

In [ ]:
# Visualize the raw dataset
train_raw_filepath = DATASET_DIR / "train.json"
!head $train_raw_filepath

{"audio_filepath": "audio/p225_200.wav", "duration": 1.864667, "text": "It looked like a river.", "speaker": 225}
{"audio_filepath": "audio/p225_264.wav", "duration": 4.483167, "text": "That should be the subject matter of a public debate.", "speaker": 225}
{"audio_filepath": "audio/p225_053.wav", "duration": 3.158875, "text": "I have made that offer.", "speaker": 225}
{"audio_filepath": "audio/p225_319.wav", "duration": 3.543479, "text": "There's still a bit to go.", "speaker": 225}
{"audio_filepath": "audio/p226_024.wav", "duration": 9.093333, "text": "This is a very common type of bow, one showing mainly red and yellow, with little or no green or blue.", "speaker": 226}
{"audio_filepath": "audio/p226_299.wav", "duration": 3.374854, "text": "There was no hint of scandal.", "speaker": 226}
{"audio_filepath": "audio/p226_326.wav", "duration": 3.418104, "text": "The airport was even different.", "speaker": 226}
{"audio_filepath": "audio/p226_252.wav", "duration": 4.611208, "text": "Ther

## Manifest Processing

The downloaded manifest is formatted for TTS training, which contains metadata such as text and speaker.

For codec training we need `audio_filepath`. The `audio_filepath` field can either be an *absolute path*, or a *relative path* with the root directory provided as an argument to each script. Here we use relative paths.

If you include `duration` the training script will automatically calculate the total size of each dataset used, and can be useful for filtering based on utterance length.

In [ ]:
def update_manifest(data_type):
    input_filepath = DATASET_DIR / f"{data_type}.json"
    output_filepath = DATASET_DIR / f"{data_type}_raw.json"

    entries = read_manifest(input_filepath)
    new_entries = []
    for entry in entries:
        # Provide relative path instead of absolute path
        audio_filepath = entry["audio_filepath"].replace("audio/", "")
        duration = round(entry["duration"], 2)
        new_entry = {
            "audio_filepath": audio_filepath,
            "duration": duration
        }
        new_entries.append(new_entry)

    write_manifest(output_path=output_filepath, target_manifest=new_entries, ensure_ascii=False)

In [ ]:
update_manifest("dev")
update_manifest("train")

In [ ]:
# Visualize updated 'audio_filepath' field.
train_filepath = DATASET_DIR / "train_raw.json"
!head $train_filepath

{"audio_filepath": "p225_200.wav", "duration": 1.86}
{"audio_filepath": "p225_264.wav", "duration": 4.48}
{"audio_filepath": "p225_053.wav", "duration": 3.16}
{"audio_filepath": "p225_319.wav", "duration": 3.54}
{"audio_filepath": "p226_024.wav", "duration": 9.09}
{"audio_filepath": "p226_299.wav", "duration": 3.37}
{"audio_filepath": "p226_326.wav", "duration": 3.42}
{"audio_filepath": "p226_252.wav", "duration": 4.61}
{"audio_filepath": "p227_019.wav", "duration": 8.79}
{"audio_filepath": "p227_354.wav", "duration": 3.59}


## Audio Preprocessing

Next we process the audio data using [preprocess_audio.py](https://github.com/NVIDIA/NeMo/blob/main/scripts/dataset_processing/tts/preprocess_audio.py).

During this step we can apply the following transformations:

1. Resample the audio from 48khz to the target sample rate for codec training.
2. Remove long silence from the beginning and end of each audio file. This can be done using an *energy* based approach which will work on clean audio, or using *voice activity detection (VAD)* which is slower but also works on audio with background or static noise (eg. from a microphone). Here we suggest VAD because some audio in VCTK has background noise.

In [ ]:
import IPython.display as ipd

In [ ]:
# Python wrapper to invoke the given bash script with the given input args
def run_script(script, args):
    args = ' \\'.join(args)
    cmd = f"python {script} \\{args}"

    print(cmd.replace(" \\", "\n"))
    print()
    !$cmd

In [ ]:
audio_preprocessing_script = NEMO_SCRIPT_DIR / "preprocess_audio.py"

# Directory with raw audio data
input_audio_dir = DATASET_DIR / "audio"
# Directory to write preprocessed audio to
output_audio_dir = DATASET_DIR / "audio_preprocessed"
# Whether to overwrite existing audio, if it exists in the output directory
overwrite_audio = True
# Whether to overwrite output manifest, if it exists
overwrite_manifest = True
# Number of threads to parallelize audio processing across
num_workers = 4
# Format of output audio files. Use "flac" to compress to a smaller file size.
output_format = "flac"
# Method for silence trimming. Can use "energy.yaml" or "vad.yaml".
trim_config_path = NEMO_CONFIG_DIR / "trim" / "vad.yaml"

def preprocess_audio(data_type):
    input_filepath = DATASET_DIR / f"{data_type}_raw.json"
    output_filepath = DATASET_DIR / f"{data_type}_manifest.json"

    args = [
        f"--input_manifest={input_filepath}",
        f"--output_manifest={output_filepath}",
        f"--input_audio_dir={input_audio_dir}",
        f"--output_audio_dir={output_audio_dir}",
        f"--num_workers={num_workers}",
        f"--output_sample_rate={SAMPLE_RATE}",
        f"--output_format={output_format}",
        f"--trim_config_path={trim_config_path}"
    ]
    if overwrite_manifest:
        args.append("--overwrite_manifest")
    if overwrite_audio:
        args.append("--overwrite_audio")

    run_script(audio_preprocessing_script, args)

In [ ]:
preprocess_audio("dev")

python /content/codec_tutorial/nemo/scripts/dataset_processing/tts/preprocess_audio.py
--input_manifest=/content/codec_tutorial/data/vctk_subset_multispeaker/dev_raw.json
--output_manifest=/content/codec_tutorial/data/vctk_subset_multispeaker/dev_manifest.json
--input_audio_dir=/content/codec_tutorial/data/vctk_subset_multispeaker/audio
--output_audio_dir=/content/codec_tutorial/data/vctk_subset_multispeaker/audio_preprocessed
--num_workers=4
--output_sample_rate=22050
--output_format=flac
--trim_config_path=/content/codec_tutorial/nemo/examples/tts/conf/trim/vad.yaml
--overwrite_manifest
--overwrite_audio

2025-01-08 17:00:00.196439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-08 17:00:00.227757: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN w

In [ ]:
preprocess_audio("train")

python /content/codec_tutorial/nemo/scripts/dataset_processing/tts/preprocess_audio.py
--input_manifest=/content/codec_tutorial/data/vctk_subset_multispeaker/train_raw.json
--output_manifest=/content/codec_tutorial/data/vctk_subset_multispeaker/train_manifest.json
--input_audio_dir=/content/codec_tutorial/data/vctk_subset_multispeaker/audio
--output_audio_dir=/content/codec_tutorial/data/vctk_subset_multispeaker/audio_preprocessed
--num_workers=4
--output_sample_rate=22050
--output_format=flac
--trim_config_path=/content/codec_tutorial/nemo/examples/tts/conf/trim/vad.yaml
--overwrite_manifest
--overwrite_audio

2025-01-08 17:00:38.209551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-08 17:00:38.296551: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuD

Before we proceed, it is important to verify that the audio processing works as expected. Let's listen to an audio file before and after processing.

Note that the processed audio is shorter because we trimmed the leading and trailing silence.

In [ ]:
!ls $processed_audio_filepath

codec_tutorial	sample_data


In [ ]:
audio_file = "p228_009.wav"
audio_filepath = input_audio_dir / audio_file
processed_audio_filepath = output_audio_dir / audio_file.replace(".wav", ".flac")

print("Original audio.")
ipd.display(ipd.Audio(audio_filepath, rate=SAMPLE_RATE))

print("Processed audio.")
ipd.display(ipd.Audio(processed_audio_filepath, rate=SAMPLE_RATE))

Original audio.


Processed audio.


# Audio Codec Training

Here we show how to train an audio codec model from scratch. Instructions and checkpoints for fine-tuning will be provided later.


In [ ]:
import os
import torch
from omegaconf import OmegaConf

In [ ]:
dataset_name = "vctk"
audio_dir = DATASET_DIR / "audio_preprocessed"
train_manifest_filepath = DATASET_DIR / "train_manifest.json"
dev_manifest_filepath = DATASET_DIR / "dev_manifest.json"

In [ ]:
audio_codec_training_script = NEMO_EXAMPLES_DIR / "audio_codec.py"

# The total number of training steps will be (epochs * steps_per_epoch)
epochs = 10
steps_per_epoch = 10

# Name of the experiment that will determine where it is saved locally and in TensorBoard and WandB
run_id = "test_run"
exp_dir = ROOT_DIR / "exps"
codec_exp_output_dir = exp_dir / MODEL_NAME / run_id
# Directory where predicted audio will be stored periodically throughout training
codec_log_dir = codec_exp_output_dir / "logs"
# Optionally log visualization of learned codes.
log_dequantized = True
# Optionally log predicted audio and other artifacts to WandB
log_to_wandb = False
# Optionally log predicted audio and other artifacts to Tensorboard
log_to_tensorboard = False

if torch.cuda.is_available():
    accelerator="gpu"
    batch_size = 4
    devices = -1
else:
    import multiprocessing
    accelerator="cpu"
    batch_size = 2
    devices = multiprocessing.cpu_count()

args = [
    f"--config-path={CONFIG_DIR}",
    f"--config-name={CONFIG_FILENAME}",
    f"max_epochs={epochs}",
    f"weighted_sampling_steps_per_epoch={steps_per_epoch}",
    f"batch_size={batch_size}",
    f"log_dir={codec_log_dir}",
    f"exp_manager.exp_dir={exp_dir}",
    f"+exp_manager.version={run_id}",
    f"model.log_config.log_wandb={log_to_wandb}",
    f"model.log_config.log_tensorboard={log_to_tensorboard}",
    f"model.log_config.generators.0.log_dequantized={log_dequantized}",
    f"trainer.accelerator={accelerator}",
    f"trainer.devices={devices}",
    f"+train_ds_meta.{dataset_name}.manifest_path={train_manifest_filepath}",
    f"+train_ds_meta.{dataset_name}.audio_dir={audio_dir}",
    f"+val_ds_meta.{dataset_name}.manifest_path={dev_manifest_filepath}",
    f"+val_ds_meta.{dataset_name}.audio_dir={audio_dir}",
    f"+log_ds_meta.{dataset_name}.manifest_path={dev_manifest_filepath}",
    f"+log_ds_meta.{dataset_name}.audio_dir={audio_dir}"
]

if MODEL_CHECKPOINT_PATH is not None:
  args.append(f"+init_from_nemo_model={MODEL_CHECKPOINT_PATH}")

In [ ]:
# If an error occurs, log the entire stacktrace.
os.environ["HYDRA_FULL_ERROR"] = "1"

In [ ]:
# Do the model training. For some configurations this step might hang when using CPU.
# ipdb.set_trace() # C - A
run_script(audio_codec_training_script, args)

python /content/codec_tutorial/nemo/examples/tts/audio_codec.py
--config-path=/content/codec_tutorial/nemo/examples/tts/conf/audio_codec
--config-name=audio_codec_low_frame_rate_22050.yaml
max_epochs=10
weighted_sampling_steps_per_epoch=10
batch_size=2
log_dir=/content/codec_tutorial/exps/AudioCodec/test_run/logs
exp_manager.exp_dir=/content/codec_tutorial/exps
+exp_manager.version=test_run
model.log_config.log_wandb=False
model.log_config.log_tensorboard=False
model.log_config.generators.0.log_dequantized=True
trainer.accelerator=cpu
trainer.devices=2
+train_ds_meta.vctk.manifest_path=/content/codec_tutorial/data/vctk_subset_multispeaker/train_manifest.json
+train_ds_meta.vctk.audio_dir=/content/codec_tutorial/data/vctk_subset_multispeaker/audio_preprocessed
+val_ds_meta.vctk.manifest_path=/content/codec_tutorial/data/vctk_subset_multispeaker/dev_manifest.json
+val_ds_meta.vctk.audio_dir=/content/codec_tutorial/data/vctk_subset_multispeaker/audio_preprocessed
+log_ds_meta.vctk.manifes

During training, the model will automatically save predictions for all audio files specified in the `log_ds_meta` manifest.

In [ ]:
codec_log_epoch_dir = codec_log_dir / "epoch_10" / dataset_name
!ls $codec_log_epoch_dir

This makes it easy to listen to the audio to determine how well the model is performing. We can decide to stop training when either:

*   The predicted audio sounds almost identical to the original audio.
*   The predicted audio stops improving in between epochs.

**Note that when training from scratch, the dataset in this tutorial is too small to get good audio quality.**

In [ ]:
audio_filepath_ground_truth = output_audio_dir / "p228_009.flac"
audio_filepath_reconstructed = codec_log_epoch_dir / "p228_009_audio_out.wav"

print("Ground truth audio.")
ipd.display(ipd.Audio(audio_filepath_ground_truth, rate=SAMPLE_RATE))

print("Reconstructed audio.")
ipd.display(ipd.Audio(audio_filepath_reconstructed, rate=SAMPLE_RATE))

dequantized_filepath = codec_log_epoch_dir / "p228_009_dequantized.png"
ipd.Image(dequantized_filepath)

# Related Information

To learn more about audio codec models in NeMo, look at our [documentation](https://docs.nvidia.com/nemo-framework/user-guide/latest/nemotoolkit/tts/models.html#codecs).

For more information on how to download and run pre-trained audio codec models, visit [NGC](https://catalog.ngc.nvidia.com/models?filters=&orderBy=scoreDESC&query=codec).

# References

1.   [EnCodec](https://arxiv.org/abs/2210.13438)
2.   [Finite Scalar Quantization (FSQ)](https://arxiv.org/abs/2309.15505)
3.   [HiFi-GAN](https://arxiv.org/abs/2010.05646)
4.   [SEANet](https://arxiv.org/abs/2009.02095)
5.   [Spectral Codecs](https://arxiv.org/abs/2406.05298)
6.   [Low Frame-rate Speech Codec](https://arxiv.org/abs/2409.12117)